## Confirm Earthdata Login

Install the [sarp-east-toolkit](https://github.com/NASA-SARP/sarp-east-toolkit).

In [ ]:
import xarray
from sarp_east_toolkit import earthdata_rio

rio_env = earthdata_rio('ornldaac')
fileobj = (
    's3://ornl-cumulus-prod-protected/'
    'gedi/GEDI_L4B_Gridded_Biomass/data/'
    'GEDI04_B_MW019MW138_02_002_05_R01000M_MU.tif'
)
with rio_env as env:
    raster = xarray.open_dataset(fileobj, engine='rasterio', chunks={})

In [ ]:
# what are the raster's dimensions? its CRS?

## Problem with XArray and Pandas

:::{dropdown} Problem
:open:

The OCO3 file has a peculiar way of storing the datetime for each sounding. The `date` variable has `epoch_dimension` as its second dimension: the 7 elements along this dimension correspond to year, month, day, hour, minute, second, and microsecond. Note that the
file name indicates all the data are from `200228`, or 2002-02-28.

Level: I

: Use the `xarray.DataArray.min` method to work out the earliest `sounding_id` time.

Level: I already knew about pandas.to_datetime

: Create a new variable in the dataset with the date converted to a datetime, getting rid of the epoch_dimension but keeping the sounding_id dimension

:::

In [ ]:
# packages you need?

In [ ]:
# file location

file = (
    DATADIR
    / 'oco-3-co2-data'
    / 'oco3_LtCO2_200228_B10400Br_220317235859s.nc4'
)

In [ ]:
# open dataset


In [ ]:
# pull the date variable out and understand this epoch_dimension
# 2020-02-28 10:36:04.1234 -> [2020, 2, 28, 10, 36, 4, 1.234]

In [ ]:
# use the `xarray.DataArray.min` method over the correct dimension


In [ ]:
# mask the date to select sounding_id with the minimum hour
# use the drop=True argument to ditch everything else


In [ ]:
# repeat above with minutes, days, ... 


In [ ]:
# how does pandas.to_datetime work when you pass it a data frame?
# how can you build a pandas.DataFrame from this xarray.DataArray?

:::{dropdown} Solution

```
from pathlib import Path

import xarray
import pandas


file = (
    Path('/efs/sarp/data/rawdata_readonly')
    / 'oco-3-co2-data'
    / 'oco3_LtCO2_200228_B10400Br_220317235859s.nc4'
)
oco3 = xarray.open_dataset(file)

date = oco3['date']
hour = date.sel({'epoch_dimension': 3}).min()
print(hour)
date = date.where(date.sel({'epoch_dimension': 3}) == hour)
minute = date.sel({'epoch_dimension': 4}).min()
print(minute)
date = date.where(date.sel({'epoch_dimension': 4}) == minute)
second = date.sel({'epoch_dimension': 5}).min()
print(second)
date = date.where(date.sel({'epoch_dimension': 5}) == second)
microsec = date.sel({'epoch_dimension': 6}).min()
print(microsec)

date = oco3['date'].assign_coords({
    'epoch_dimension': ['year', 'month', 'day', 'hour', 'minute', 'second', 'microsecond']
})
date_pandas = date.to_dataset(dim='epoch_dimension').to_dataframe()
oco3['datetime'] = pandas.to_datetime(date_pandas)
```
:::